In [1]:
from utils.fid import FrechetInceptionDistance
from models.conv_generator import ConvGenerator
from data_loaders.mnist import MnistDataLoaderFactory
from dotmap import DotMap
from tqdm.notebook import tqdm
import os

In [2]:
config = DotMap()
config.data = DotMap(root="/tmp/data", batch_size=128)
config.device = "cpu"

In [3]:
fid = FrechetInceptionDistance(config.device)
dl = MnistDataLoaderFactory.get_data_loader(config)

In [4]:
model_files = [f"gen{i}.p" for i in range(1, 4)]
models = []
for fname in model_files:
    model = ConvGenerator(config)
    model.load(os.path.join("trained_models", fname), map_location="cpu")
    model.eval()
    models.append(model)

Get 20 batches (about 1200 samples)

In [5]:
di = iter(dl)
batches = [next(di) for _ in range(20)]

In [ ]:
scores = []
for model, path in zip(models, model_files):
    for X_true in tqdm(batches):
        X_fake = model.generate_batch(X_true.shape[0], config.device)
        fid.add_batch(X_true, X_fake)
    score = fid.calculate()
    print(f"{path}: {score:.4f}")